In [ ]:
#default_exp core

In [ ]:
#export
from fastcore.utils import *
from fastcore.foundation import *
from fastcore.meta import *

import pprint,inspect,json,copy
from inspect import signature,Parameter,Signature
from urllib.parse import urlencode
from ghapi.metadata import funcs
from urllib.request import Request

In [ ]:
#export
GH_HOST = "https://api.github.com"
_DOC_URL = 'https://docs.github.com/'

In [ ]:
def _mk_param(nm, **kwargs): return Parameter(nm, kind=Parameter.POSITIONAL_OR_KEYWORD, **kwargs)

def mk_sig(req_args, def_args):
    params =  [_mk_param(k) for k in req_args]
    params += [_mk_param(k, default=v) for k,v in def_args.items()]
    return Signature(params)

In [ ]:
#export
class GhVerb:
    __slots__ = 'path,verb,tag,name,summary,url,route_ps,params,data,hdrs,__doc__'.split(',')
    def __init__(self, path, verb, oper, summary, url, params, data, hdrs, kwargs):
        tag,name = oper.split('/')
        name = name.replace('-','_')
        path,route_ps,xtra = partial_format(path, **kwargs)
        __doc__ = summary
        store_attr()
    
    def __call__(self, *args, **kwargs):
        flds = [o for o in self.route_ps+self.params+self.data if o not in kwargs]
        for a,b in zip(args,flds): kwargs[b]=a
        route_p,query_p,data_p = [{p:kwargs[p] for p in o if p in kwargs}
                                 for o in (self.route_ps,self.params,self.data)]
        return dict2obj(urlsend(GH_HOST+self.path, self.verb, headers=self.hdrs,
                              route=route_p, query=query_p, data=data_p))

    @property
    def __signature__(self): return mk_sig(self.route_ps, {})
    __call__.__signature__ = __signature__

    def _repr_markdown_(self):
        params = ','.join(self.params+self.data)
        return f"[{self.tag}/{self.name}]({_DOC_URL}{self.url})({params}): {self.summary}"
    __repr__ = _repr_markdown_

class _GhVerbGroup(AttrDict):
    def __init__(self, verbs): super().__init__(**{o.name:o for o in verbs})
    def _repr_markdown_(self): return "\n".join(f'- [{k}]({_DOC_URL}{v.url})' for k,v in self.items())

In [ ]:
#export
class GhApi:
    def __init__(self, owner=None, repo=None, token=None, **kwargs):
        self._headers = { 'Accept': 'application/vnd.github.v3+json' }
        if token: self._headers['Authorization'] = 'token ' + token
        if owner: kwargs['owner'] = owner
        if repo:  kwargs['repo' ] = repo
        funcs_ = L(funcs).starmap(GhVerb, hdrs=self._headers, kwargs=kwargs)
        self._fs = {k:_GhVerbGroup(v) for k,v in groupby(funcs_, 'tag').items()}

    def __dir__(self): return super().__dir__() + list(self._fs)

    def __getattr__(self,k):
        if k in self._fs: return self._fs[k]
        raise AttributeError(k)

In [ ]:
api = GhApi('fastai', 'fastcore', os.environ['FASTRELEASE_TOKEN'])

In [ ]:
api.git.get_ref('heads/master')

- ref: refs/heads/master
- node_id: MDM6UmVmMjI1NDYwNTk5OnJlZnMvaGVhZHMvbWFzdGVy
- url: https://api.github.com/repos/fastai/fastcore/git/refs/heads/master
- object: 
  - sha: 88be5bd61f8b476778473dad509f0d3060862ae8
  - type: commit
  - url: https://api.github.com/repos/fastai/fastcore/git/commits/88be5bd61f8b476778473dad509f0d3060862ae8

In [ ]:
api.git.get_ref

[git/get_ref](https://docs.github.com/rest/reference/git#get-a-reference)(): Get a reference

In [ ]:
api.git.get_ref.path

'/repos/{owner}/{repo}/git/ref/{ref}'

In [ ]:
api.git

- [create_blob](https://docs.github.com/rest/reference/git#create-a-blob)
- [get_blob](https://docs.github.com/rest/reference/git#get-a-blob)
- [create_commit](https://docs.github.com/rest/reference/git#create-a-commit)
- [get_commit](https://docs.github.com/rest/reference/git#get-a-commit)
- [list_matching_refs](https://docs.github.com/rest/reference/git#list-matching-references)
- [get_ref](https://docs.github.com/rest/reference/git#get-a-reference)
- [create_ref](https://docs.github.com/rest/reference/git#create-a-reference)
- [update_ref](https://docs.github.com/rest/reference/git#update-a-reference)
- [delete_ref](https://docs.github.com/rest/reference/git#delete-a-reference)
- [create_tag](https://docs.github.com/rest/reference/git#create-a-tag-object)
- [get_tag](https://docs.github.com/rest/reference/git#get-a-tag)
- [create_tree](https://docs.github.com/rest/reference/git#create-a-tree)
- [get_tree](https://docs.github.com/rest/reference/git#get-a-tree)

```python
its = 'fastai,fastgpu,fastcore,fastprogress,fastrelease,nbdev,ghapi'.split(',')
url = 'https://webhooks.fast.ai'
for it in its:
    api.repo = it
    for hook in api.repos.list_webhooks(): api.repos.delete_webhook(hook.id)
    cfg = dict(url=url, content_type='json', secret='XXX')
    api.repos.create_webhook(config=cfg, events=['release'])
```

In [ ]:
def _tag_date(self, tag):
    try: tag_d = self.gh(f"git/ref/tags/{tag}")
    except HTTPError: raise Exception(f"Failed to find tag {tag}")
    commit_d = self.gh(tag_d.object.url, complete=True)
    self.commit_date = commit_d.committer.date
    return self.commit_date

In [ ]:
def release(self):
    "Tag and create a release in GitHub for the current version"
    run(f'git tag {ver}')
    run('git push --tags')
    run('git pull --tags')
    self.gh("releases", post=True, tag_name=ver, name=ver, body=notes)

## Export -

In [ ]:

#hide
from nbdev.export import notebook2script
notebook2script()

Converted 00_core.ipynb.
Converted 90_build_lib.ipynb.
Converted index.ipynb.
